In [1]:
#main_taberconley_controls.do
#sample.do
#!pip install linearmodels
#!pip install econtools
import pandas as pd
import numpy as np
import matplotlib as plt
from linearmodels import PanelOLS
import statsmodels.api as sm
import econtools as econ
import econtools.metrics as mt

df = pd.read_stata('data/Authority.dta')
print(df.shape)
#list(df.columns)

#construct work category dummy
df['OG03_dummy'] = 0
df.loc[(df['work_category']=='OG03')&(df['work_category']!=''),'OG03_dummy'] = 1

df['OG01_dummy'] = 0
df.loc[(df['work_category']=='OG01')&(df['work_category']!=''),'OG01_dummy'] = 1

df['OG_rest_dummy'] = 0
df.loc[(df['OG01_dummy']!=1)&(df['OG03_dummy']!=1)&(df['work_category']!=''),'OG_rest_dummy'] = 1

df['OG_dummy'] = 0
df.loc[df['work_category'].str[0:2] == 'OG','OG_dummy'] = 1

df['OS_dummy'] = 0
df.loc[df['work_category'].str[0:2] == 'OS','OS_dummy'] = 1

#treated vs controls
df['trend'] = df['year'] - 1999

df['trend_treat'] = df['trend']
df.loc[(df['authority_code']!=3090272)&(df['authority_code']!=3070001),'trend_treat'] = 0
#15225 real change made / no zeros in df['trend']
#print(df['trend_treat'].value_counts()) chekced

df['trend_control'] = df['trend']
df.loc[(df['authority_code']==3090272)|(df['authority_code']==3070001),'trend_control'] = 0
#902 real change made
#print(df['trend_control'].value_counts()) checked

#PA specifics
df = df.sort_values(by='authority_code',ascending=True)
#auth = econ.group_id(df, cols = 'authority_code')
#print(auth) #dataframe

auth_list = df['authority_code'].values.tolist()
auth_list = list(set(auth_list))

#겹치는 부분 = authority_code야
#id_auth = group_id + 1 if df의 auth code == auth의 code

df['id_auth'] = 0
for i in range(len(df)):
    for j in range(len(auth_list)):
        if df.loc[i,'authority_code'] == auth_list[j]:
            df.loc[i,'id_auth'] = j+1
#id_auth =  econ.group_id(df, cols='authority_code')
#print(id_auth)
#print(df['id_auth'].values.tolist())

work_dum = pd.get_dummies(df['work_category'])
year_dum = pd.get_dummies(df['year'])
work_list = list(work_dum.columns)
year_list = list(year_dum.columns)

df_dum = pd.concat([year_dum, work_dum],axis = 1)
df = pd.concat([df, df_dum],axis = 1)

#for statement check            
#df.head
#print(max(df['id_path']))

(16127, 31)


In [122]:
#keep the observations need
#df = untouched 
df1 = df
df1 = df1[((df1['turin_co_sample'] == 1)|(df1['turin_pr_sample'] == 1))&((df1['post_experience']>=5)|(df1['post_experience'].isnull()==True)) &((df1['pre_experience']>=5)|(df1['pre_experience'].isnull()==True)) &(df1['missing']==0)]
#df_pr = df1[(df1['turin_co_sample'] != 1)&(df1['turin_pr_sample'] == 1)&(df1['post_experience']>=5) &(df1['pre_experience']>=5) &(df1['missing']==0)]
#df_co_pr = df1[(df1['turin_co_sample'] == 1)&(df1['turin_pr_sample'] == 1)&(df1['post_experience']>=5) &(df1['pre_experience']>=5) &(df1['missing']==0)]
#df1 = pd.concat([df_co,df_pr,df_co_pr])
df1= df1[(df1['ctrl_pop_turin_co_sample']==1) | (df1['ctrl_pop_turin_pr_sample']==1) | (df1['ctrl_exp_turin_co_sample']==1) | (df1['ctrl_exp_turin_pr_sample']==1) | (df1['ctrl_pop_exp_turin_co_sample']==1) | (df1['ctrl_pop_exp_turin_pr_sample']==1)]
print(df1.shape)

(2016, 94)


In [123]:
#reconstruct trend_pa (another version)
id_auth_group =  econ.group_id(df1, cols='id_auth')
id_auth_group['group_id'] = id_auth_group['group_id'] +1
#print(id_auth_group)

df1['id_auth_remained'] = 0
for i in range(len(df1)):
    for j in range(len(id_auth_group)):
        if df1.iloc[i,df1.columns.get_loc('id_auth')] == id_auth_group.loc[j, 'id_auth']:
            df1.iloc[i,df1.columns.get_loc('id_auth_remained')] = id_auth_group.loc[j, 'group_id']
id_auth_remained_dum = pd.get_dummies(df1['id_auth_remained']).rename(columns=lambda x: 'id_auth_remained' + str(x))
df1 = pd.concat([df1, id_auth_remained_dum],axis = 1)

for i in range(len(id_auth_group)):
    df1['trend_pa_remained'+str(i+1)] =0
    col_num_tr = df1.columns.get_loc('trend_pa_remained'+str(i+1))
    col_num_id = df1.columns.get_loc('id_auth_remained'+str(i+1))
    for j in range(len(df1)):
        if df1.iloc[j, col_num_id]==1 and df1.iloc[j, df1.columns.get_loc('authority_code')]!= 3090272 and df1.iloc[j,df1.columns.get_loc('authority_code')]!=3070001:
            df1.iloc[j ,col_num_tr] = df1.iloc[j, df1.columns.get_loc('trend')]
    df1.drop(['id_auth_remained'+str(i+1)], axis = 1)

#temp location working check

In [124]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

# mutlicollinearity function
def calc_vif(X):
    vif = pd.DataFrame()
    vif["variables"] = X.columns
    vif["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

    return(vif)

treatment = ['turin_co_sample'] #, 'turin_pr_sample']
group = [ 'ctrl_exp'] #, 'ctrl_pop', 'ctrl_pop_exp']
outcomes = ['discount'] #, 'delay_ratio', 'overrun_ratio', 'days_to_award'] 
c_outcomes = 1
i = 5
df1['dummy_cat'] =0

#create dummies pa-year(for statement)

for t in treatment:
    for g in group:
        for o in outcomes:
            #matrix = nc_ + o + t 
            df1 = df1[(df1[t]==1) & (df1[g +'_' + t]==1) & (df1['post_experience']>=i) & (df1['pre_experience']>=i)& (df1['post_experience'].isnull()==False) & (df1['pre_experience'].isnull()==False) & (df1['missing']==0) & (df1[o].isnull()==False) & (df1['fiscal_efficiency'].isnull()==False) & (df1['reserve_price'].isnull()==False)&(df1['municipality'].isnull()==False)]
            df1.sort_values(by = 'authority_code', ascending = True)
            df1['ind'] = np.nan
            for i in range(len(df1)):
                if df1.iloc[i, df1.columns.get_loc('authority_code')] != df1.iloc[i-1, df1.columns.get_loc('authority_code')]:
                    df1.iloc[i, df1.columns.get_loc('ind')] = 1
            
            #create dummies for administration-year pairs 
            all_years = df1['year'].unique()
            all_authorities = df1['authority_code'].unique()
            auth_year_reg_col = []
            for auth in all_authorities:
                for yr in all_years:
                    df1['auth_year_' + str(int(auth))+'_' + str(int(yr))] = 0
                    auth_year_reg_col.append('auth_year_' + str(int(auth))+'_' + str(int(yr)))
                    df1.loc[(df1['year']==yr) & (df1['authority_code']==auth), 'auth_year_' + str(int(auth))+'_' + str(int(yr)) ] = 1
           
            # create dummies for work category
            all_categories = df1['work_category'].unique()
            i = 1
            for cat in all_categories:
                df1['cat_'+cat] = 0
                df1.loc[df1['work_category']==cat, 'cat_'+cat] =1
                if i == 1:
                #line91
                    i = i+1
                #else:
                #line 95
       
            ### Regression first stage 
            #make a exog list
            reg_col = []
            for i in work_list:
                reg_col.append(i)
            for j in year_list:
                reg_col.append(j)
            
            exog_var = ['fpsb_auction','reserve_price','municipality']
            exog = exog_var + reg_col 
            #i_authority_code 안 만드는 이유 = fe_name에 들어가있어서

            #vif cal
            #check multicollinearity
            X = df1.loc[:,exog]
            vif = calc_vif(X)
            #print(vif)


            #delete from col list
            for i in range(len(vif)):
                if np.isnan(vif.loc[i, 'VIF']) == True:
                    reg_col.remove(vif.loc[i, 'variables'])
                elif vif.loc[i,'VIF'] > 10:
                    for j in exog_var:
                        if str(vif.loc[i,'variables']) is j and vif.loc[i,'variables'] is not 'fpsb_auction' and vif.loc[i,'variables'] is not 'id_auth':
                            exog_var.remove(vif.loc[i,'variables'])
                
            exog = exog_var + reg_col
            exog.remove(2000)
            exog.remove('OG01')
            exog.remove('municipality')
    
            #1. reg
            fe_reg_1 = mt.reg(df1, o, exog, fe_name = 'authority_code', cluster = 'auth_anno')
            
            #2. reg
            fe_reg_2 = mt.reg(df1, o, exog, fe_name = 'authority_code', cluster = 'authority_code')
            
            #3. reg
            reg_col = auth_year_reg_col
            for cat in all_categories:
                reg_col.append('cat_'+cat)
            exog_var = ['reserve_price','municipality','fiscal_efficiency']
            exog = exog_var + reg_col
            
            X = df1.loc[:,exog]
            vif = calc_vif(X)
            #print(vif)
            for i in range(len(vif)):
                if np.isnan(vif.loc[i, 'VIF']) == True:
                    reg_col.remove(vif.loc[i, 'variables'])
                '''elif vif.loc[i,'VIF'] > 10:
                    for j in exog:
                        exog_var.remove(vif.loc[i,'variables'])'''
                
            exog = exog_var + reg_col
            exog.remove('municipality')
            
            fe_reg_3 = mt.reg(df1, o, exog, cluster = 'auth_anno')
            #print(fe_reg_1, fe_reg_2, fe_reg_3)
            
           
            #d["{0}_hat".format(o)] =fe_reg_3.yhat
            if o == 'discount':
                discount_hat = fe_reg_3.yhat
                for cat in all_categories:
                    df1['discount_beta'] = discount_hat - (df1['dummy_cat']-df1['cat_'+cat] * fe_reg_3.beta['cat_'+cat]) - df1['reserve_price']*fe_reg_3.beta['reserve_price'] -df1['fiscal_efficiency']*fe_reg_3.beta['fiscal_efficiency'] #-df1['municipality']*fe_reg_3.beta['municipality']
            elif o == 'delay_ratio':
                delay_ratio_hat = fe_reg_3.yhat
                for cat in all_categories:
                    df1['delay_ratio_beta'] = delay_ratio_hat - (df1['dummy_cat']-df1['cat_'+cat] * fe_reg_3.beta['cat_'+cat]) - df1['reserve_price']*fe_reg_3.beta['reserve_price'] -df1['fiscal_efficiency']*fe_reg_3.beta['fiscal_efficiency'] # -df1['municipality']*fe_reg_3.beta['municipality']
            elif o == 'overrun_ratio':
                overrun_ratio_hat = fe_reg_3.yhat
                for cat in all_categories:
                    df1['overrun_ratio_beta'] = overrun_ratio_hat - (df1['dummy_cat']-df1['cat_'+cat] * fe_reg_3.beta['cat_'+cat]) - df1['reserve_price']*fe_reg_3.beta['reserve_price'] -df1['fiscal_efficiency']*fe_reg_3.beta['fiscal_efficiency'] # -df1['municipality']*fe_reg_3.beta['municipality']
            else:
                days_to_award_hat = fe_reg_3.yhat
                for cat in all_categories:
                    df1['days_to_award_beta'] =days_to_award_hat - (df1['dummy_cat']-df1['cat_'+cat] * fe_reg_3.beta['cat_'+cat]) - df1['reserve_price']*fe_reg_3.beta['reserve_price'] -df1['fiscal_efficiency']*fe_reg_3.beta['fiscal_efficiency'] #-df1['municipality']*fe_reg_3.beta['municipality']
            #worked
            
            #create weigths - working well
            nrep_s = df1.groupby(['authority_code','year']).size().unstack(level=1)
            df1_nrep = pd.DataFrame(nrep_s)/len(df1)
            df1['weights'] = np.nan
            for auth in all_authorities:
                for yr in all_years:
                    df1.loc[(df1['authority_code']==auth)&(df1['year']==yr),'weights'] = df1_nrep.loc[auth, yr]
            
            #Keep only beta coefficients for state*year terms
            collapse = df1.groupby(['auth_anno'])[o +'_beta', 'authority_code', 'year', 'fpsb_auction', 'municipality', 'fiscal_efficiency', 'missing', 'turin_co_sample', 'weights'].mean()
            #어디에 반영되는 거지? merge해야하나?
            
            #Core conley-taber method
            exog_var = ['fpsb_auction', 'municipality', 'fiscal_efficiency']
            reg_col = []
            for j in year_list:
                reg_col.append(j)
            
            X = df1.loc[:,exog]
            vif = calc_vif(X)

            #delete from col list
            for i in range(len(vif)):
                if np.isnan(vif.loc[i, 'VIF']) == True:
                    reg_col.remove(vif.loc[i, 'variables'])
                elif vif.loc[i,'VIF'] > 10:
                    for j in exog_var:
                        if str(vif.loc[i,'variables']) is j and vif.loc[i,'variables'] is not 'fpsb_auction' and vif.loc[i,'variables'] is not 'id_auth':
                            exog_var.remove(vif.loc[i,'variables'])
                
            exog = exog_var + reg_col
            #exog.remove(2000)
            #exog.remove('municipality')
            
            wls = mt.reg(df1, o+'_beta', reg_col , fe_name = 'authority_code', cluster = 'auth_anno',awt_name = 'weights', check_colinear = True)


C:\Users\thxkn\anaconda3\lib\site-packages\ipykernel_launcher.py:144: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.


TypeError: sequence item 0: expected str instance, float found

In [97]:
nrep_s = df1.groupby(['authority_code','year']).size().unstack(level=1)
print(nrep_s)
df1_nrep = pd.DataFrame(nrep_s)/len(df1)


year            2000.0  2001.0  2002.0  2003.0  2004.0  2005.0  2006.0
authority_code                                                        
3.0                2.0    12.0    13.0    19.0     8.0    16.0     4.0
4.0                NaN     2.0    10.0    12.0    12.0     2.0     4.0
6.0                1.0     3.0     5.0     8.0     5.0     2.0     4.0
9.0               19.0    13.0    20.0     3.0    27.0     8.0     4.0
16.0              12.0    27.0     NaN     NaN     NaN     7.0    15.0
20.0               2.0     1.0     6.0     3.0    12.0    13.0    11.0
25.0               8.0     4.0     8.0     5.0    11.0    11.0     4.0
30.0               1.0     2.0     9.0    11.0    17.0    12.0     5.0
1246.0             1.0     5.0    20.0     8.0    16.0    19.0    13.0
1708.0             4.0    17.0    12.0    14.0     6.0    11.0     2.0
1739.0             8.0    14.0    22.0    26.0    23.0    12.0    14.0
1768.0             5.0    14.0    21.0    27.0    29.0    27.0    20.0
1858.0

year,2000.0,2001.0,2002.0,2003.0,2004.0,2005.0,2006.0
authority_code,,,,,,,
3.0,0.001585,0.009509,0.010301,0.015055,0.006339,0.012678,0.003170
4.0,NaN,0.001585,0.007924,0.009509,0.009509,0.001585,0.003170
6.0,0.000792,0.002377,0.003962,0.006339,0.003962,0.001585,0.003170
9.0,0.015055,0.010301,0.015848,0.002377,0.021395,0.006339,0.003170
16.0,0.009509,0.021395,NaN,NaN,NaN,0.005547,0.011886


In [115]:
str(200.0)

'200.0'

In [40]:
#sm.ols
from statsmodels.stats.outliers_influence import variance_inflation_factor
# mutlicollinearity function
def calc_vif(X):
    vif = pd.DataFrame()
    vif["variables"] = X.columns
    vif["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

    return(vif)
treatment = ['turin_co_sample'] #, 'turin_pr_sample']
group = [ 'ctrl_exp'] #, 'ctrl_pop', 'ctrl_pop_exp']
outcomes = ['discount'] #, 'delay_ratio', 'overrun_ratio', 'days_to_award'] 
c_outcomes = 1
i = 5
d= {}

for t in treatment:
    for g in group:
        for o in outcomes:
            #matrix = nc_ + o + t 
            df1 = df1[(df1[t]==1) & (df1[g +'_' + t]==1) & (df1['post_experience']>=i) & (df1['pre_experience']>=i)& (df1['post_experience'].isnull()==False) & (df1['pre_experience'].isnull()==False) & (df1['missing']==0) & (df1[o].isnull()==False) & (df1['fiscal_efficiency'].isnull()==False) & (df1['reserve_price'].isnull()==False)&(df1['municipality'].isnull()==False)]
            df1.sort_values(by = 'authority_code', ascending = True)
            #for j in range(len(df_tab5)):
            #    if df_tab5.loc[j,'authority_code']!=df[[:,j],'authority_code']:
            #        df_tab5['ind'] = 1
            #line61-64 skip
            
            #create dummies for administration-year pairs 
            all_years = df1['year'].unique()
            all_authorities = df1['authority_code'].unique()
            auth_year_reg_col = []
            for auth in all_authorities:
                for yr in all_years:
                    df1['auth_year_' + str(int(auth))+'_' + str(int(yr))] = 0
                    auth_year_reg_col.append('auth_year_' + str(int(auth))+'_' + str(int(yr)))
                    df1.loc[(df1['year']==yr) & (df1['authority_code']==auth), 'auth_year_' + str(int(auth))+'_' + str(int(yr)) ] = 1
           
            # create dummies for work category
            all_categories = df1['work_category'].unique()
            i = 1
            for cat in all_categories:
                df1['cat_'+cat] = 0
                df1.loc[df1['work_category']==cat, 'cat'+cat] =1
                if i == 1:
                #line91
                    i = i+1
                #else:
                #line 95
       
            ### Regression first stage 
            #make a exog list
            reg_col = []
            for i in work_list:
                reg_col.append(i)
            for j in year_list:
                reg_col.append(j)
            
            exog_var = ['fpsb_auction','reserve_price','municipality']
            exog = exog_var + reg_col 
            #i_authority_code 안 만드는 이유 = fe_name에 들어가있어서

            #vif cal
            #check multicollinearity
            X = df1.loc[:,exog]
            vif = calc_vif(X)
            #print(vif)


            #delete from col list
            for i in range(len(vif)):
                if np.isnan(vif.loc[i, 'VIF']) == True:
                    reg_col.remove(vif.loc[i, 'variables'])
                elif vif.loc[i,'VIF'] > 10:
                    for j in exog_var:
                        if str(vif.loc[i,'variables']) is j and vif.loc[i,'variables'] is not 'fpsb_auction' and vif.loc[i,'variables'] is not 'id_auth':
                            exog_var.remove(vif.loc[i,'variables'])
                
            exog = exog_var + reg_col
            exog.remove(2000)
            exog.remove('OG01')
            exog.remove('municipality')
            
            y = df1[o]
            x = df1[exog]
    
            #1. reg
            fe_reg_1 = sm.OLS(y, x).fit(cov_type='cluster', cov_kwds={'groups': df1['auth_anno']})
            #fe_name = 'authority_code', cluster = 'auth_anno')
            

            
            
            #2. reg
            fe_reg_2 = mt.reg(df1, o, exog, fe_name = 'authority_code', cluster = 'authority_code')
            
            #3. reg
            reg_col = auth_year_reg_col
            for cat in all_categories:
                reg_col.append('cat_'+cat)
            exog_var = ['reserve_price','municipality','fiscal_efficiency']
            exog = exog_var + reg_col
            
            X = df1.loc[:,exog]
            vif = calc_vif(X)
            #print(vif)
            for i in range(len(vif)):
                if np.isnan(vif.loc[i, 'VIF']) == True:
                    reg_col.remove(vif.loc[i, 'variables'])
                '''elif vif.loc[i,'VIF'] > 10:
                    for j in exog:
                        exog_var.remove(vif.loc[i,'variables'])'''
                
            exog = exog_var + reg_col
            exog.remove('municipality')
            
            fe_reg_3 = mt.reg(df1, o, exog, cluster = 'auth_anno')
            
            print(fe_reg_1.summary(),fe_reg_2, fe_reg_3)
           
            d["{0}_hat".format(o)] =fe_reg_3.yhat

C:\Users\thxkn\anaconda3\lib\site-packages\statsmodels\stats\outliers_influence.py:193: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)
C:\Users\thxkn\anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1685: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - self.ssr/self.centered_tss


                                 OLS Regression Results                                
Dep. Variable:               discount   R-squared (uncentered):                   0.861
Model:                            OLS   Adj. R-squared (uncentered):              0.857
Method:                 Least Squares   F-statistic:                              303.0
Date:                Mon, 06 Jul 2020   Prob (F-statistic):                    1.17e-85
Time:                        20:57:49   Log-Likelihood:                         -4034.6
No. Observations:                1262   AIC:                                      8143.
Df Residuals:                    1225   BIC:                                      8333.
Df Model:                          37                                                  
Covariance Type:              cluster                                                  
                    coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------

C:\Users\thxkn\anaconda3\lib\site-packages\econtools\metrics\core.py:203: RuntimeWarning: invalid value encountered in sqrt
  se = pd.Series(np.sqrt(np.diagonal(vce)), index=vce.columns)
C:\Users\thxkn\anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
C:\Users\thxkn\anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
C:\Users\thxkn\anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1827: RuntimeWarning: invalid value encountered in greater_equal
  cond2 = (x >= np.asarray(_b)) & cond0
C:\Users\thxkn\anaconda3\lib\site-packages\statsmodels\base\model.py:1832: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 37, but rank is 31
  'rank is %d' % (J, J_), ValueWarning)


In [38]:
fe_reg_1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:               discount   R-squared (uncentered):                   0.861
Model:                            OLS   Adj. R-squared (uncentered):              0.857
Method:                 Least Squares   F-statistic:                              303.0
Date:                Mon, 06 Jul 2020   Prob (F-statistic):                    1.17e-85
Time:                        20:56:39   Log-Likelihood:                         -4034.6
No. Observations:                1262   AIC:                                      8143.
Df Residuals:                    1225   BIC:                                      8333.
Df Model:                          37                                                  
Covariance Type:              cluster                                                  
=================================================================================
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
fpsb_auction     17.4478      0.862     20.240      0.000      15.758      19.137
reserve_price  3.063e-06   5.27e-07      5.815      0.000    2.03e-06     4.1e-06
OG02              2.5951      1.061      2.447      0.014       0.516       4.674
OG03              2.6695      1.308      2.041      0.041       0.106       5.233
OG04             -2.6530      3.032     -0.875      0.382      -8.595       3.289
OG06              4.1959      1.350      3.108      0.002       1.550       6.842
OG07              7.0969      5.364      1.323      0.186      -3.416      17.610
OG08              5.3317      2.465      2.163      0.031       0.500      10.164
OG10             12.4312      1.220     10.187      0.000      10.040      14.823
OG11              8.0753      1.520      5.313      0.000       5.096      11.054
OG12             10.2903      2.627      3.917      0.000       5.142      15.439
OS01             -1.8358      2.681     -0.685      0.493      -7.090       3.418
OS02              9.5425      1.458      6.545      0.000       6.685      12.400
OS05             15.4102      0.537     28.723      0.000      14.359      16.462
OS06              7.2794      1.723      4.224      0.000       3.902      10.657
OS07              6.1595      1.272      4.842      0.000       3.666       8.653
OS08             10.6023      5.828      1.819      0.069      -0.821      22.025
OS10             16.5178      1.978      8.353      0.000      12.642      20.394
OS11             -7.1741      1.752     -4.094      0.000     -10.608      -3.740
OS12             20.0592      1.618     12.401      0.000      16.889      23.229
OS14              5.9653      1.357      4.396      0.000       3.306       8.625
OS18              3.8815      3.103      1.251      0.211      -2.200       9.963
OS19              7.0820      1.170      6.055      0.000       4.789       9.374
OS21              9.6818      2.982      3.247      0.001       3.838      15.526
OS22              7.9852      2.001      3.992      0.000       4.064      11.906
OS23              4.7119      3.261      1.445      0.149      -1.680      11.104
OS24              4.5222      2.205      2.050      0.040       0.200       8.845
OS26              6.2795      1.715      3.661      0.000       2.918       9.641
OS28              7.9940      1.572      5.084      0.000       4.912      11.076
OS30             10.0759      1.542      6.535      0.000       7.054      13.098
OS34             13.3993      2.768      4.841      0.000       7.974      18.825
2001.0            7.9771      1.859      4.290      0.000       4.333      11.621
2002.0            6.0173      1.553      3.874      0.000       2.973       9.062
2003.0            5.3006      1.

In [ ]:
#old code already merged
# create dummies for work category
all_categories = df['work_cateogory'].unique()

from statsmodels.stats.outliers_influence import variance_inflation_factor

i = 1
for cat in all_categories:
    df['cat_'+cat] = 0
    df.loc[df['work_category']==cat, 'cat'+cat] =1
    if i == 1:
        #line91
        i = i+1
    else:
        #line 95
        
# Regression first stage 
    # mutlicollinearity function
    def calc_vif(X):

        # Calculating VIF
        vif = pd.DataFrame()
        vif["variables"] = X.columns
        vif["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

        return(vif)

    #iteration
#for o in outcome: create dum pa-year이랑 연결됨
    #days_to_award(4,6)
    idx = df_reg_co[df_reg_co[o].isnull()==True].index
    df_name = df_reg_co.drop(idx)
    
    #vif cal
    #first, make a column list
    reg_col = []
    for i in work_list:
        reg_col.append(i)
    for j in year_list:
        reg_col.append(j)
        
        reg_col.append('auth_year_' + auth + yr)
    #i_authority_code 안 만드는 이유 = fe_name에 들어가있어서
    exog_var = ['fpsb_auction','reserve_price','municipality']
    exog = exog_var + reg_col 


    #check multicollinearity
    X = df_name.loc[:,exog]
    vif = calc_vif(X)
    #print(vif)


    #delete from col list
    for i in range(len(vif)):
        if np.isnan(vif.loc[i, 'VIF']) == True:
            reg_col.remove(vif.loc[i, 'variables'])
        elif vif.loc[i,'VIF'] > 10:
            for j in exog_var:
                if str(vif.loc[i,'variables']) is j and vif.loc[i,'variables'] is not 'fpsb_auction' and vif.loc[i,'variables'] is not 'id_auth':
                    exog_var.remove(vif.loc[i,'variables'])
                
    exog = exog_var + reg_col
    exog.remove(2000)
    exog.remove('OG01')
    #exog.remove('municipality')
    
    #1. reg
    fe_reg = mt.reg(df_name, o, exog, fe_name = 'authority_code', cluster = 'auth_anno')
    #print(fe_reg) 
    if 'c_outcomes' == 1:
        #outreg2
    else:
        #outreg2
        
    #2. reg cluster(authority_code) line110
    fe_reg = mt.reg(df_name, o, exog, fe_name = 'authority_code', cluster = 'authority_code')
    if 'c_outcomes' == 1:
        #outreg
    else: 
        #outreg
    
    
    
    ####3. reg line 118
    def calc_vif(X):

        # Calculating VIF
        vif = pd.DataFrame()
        vif["variables"] = X.columns
        vif["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

        return(vif)

    #iteration
    #days_to_award(4,6)
    idx = df_reg_co[df_reg_co[o].isnull()==True].index
    df_name = df_reg_co.drop(idx)
    
    #vif cal
    #first, make a column list
    reg_col = []
    for i in all_categories:
        reg_col.append('cat_'+i)
    for auth in all_authorities:
        for yr in all_years:
             reg_col.append('auth_year_' + auth + yr)
    
    exog_var = ['fiscal efficiency','reserve_price','municipality']
    exog = exog_var + reg_col 


    #check multicollinearity
    X = df_name.loc[:,exog]
    vif = calc_vif(X)
    #print(vif)


    #delete from col list
    for i in range(len(vif)):
        if np.isnan(vif.loc[i, 'VIF']) == True:
            reg_col.remove(vif.loc[i, 'variables'])
        elif vif.loc[i,'VIF'] > 10:
            for j in exog_var:
                if str(vif.loc[i,'variables']) is j and vif.loc[i,'variables'] is not 'fpsb_auction' and vif.loc[i,'variables'] is not 'id_auth':
                    exog_var.remove(vif.loc[i,'variables'])
                
    exog = exog_var + reg_col
    #exog.remove('municipality')
    
    #1. reg
    fe_reg = mt.reg(df_name, o, exog, cluster = 'auth_anno')
    
#line 120-121

In [ ]:
#create weights (123)


In [ ]:
#* Keep only beta coefficients for state*year terms *(130)

In [ ]:
#* Core conley-taber method*

In [ ]:
#/* Predict residuals from regression */(137)

In [ ]:
#/* Create d tilde variable*/
df_tab5 = df_tab5.sort_values(by = 'year', ascending = True)
df_tab5.loc[(df['authority_code']==3090272) | (df['authority_code']==3070001),'djtga'] = df['authority_code'].mean()
df_tab5['djt'] = df['djtga'].sum()

df_tab5 = df_tab5.sort_values(by = 'authority_code',ascending = True)
df_tab5['meandjt'] = df['djt'].mean()
df_tab5['tilde'] = df['djt'] - df['meandjt']

In [ ]:
#line148 Obtain difference in differences coefficient*/
#line149 re-normalize weights*/

df.loc[(df['authority_code']==3090272) | (df['authority_code']==3070001), 'tot_weights'] = df['weights']
df['new_weights'] = df['weights']/df['tot_weights']
mod_wls = sm.WLS(eta, dtil, weights=df['new_weights']).fit()
df.drop(['new_weights','tot_weights'])

In [ ]:
#/* Simulations for each public administration

#line160
for auth in all_authorities:


#line 177,178

In [ ]:
# Confidence intervals

#line 231